# Model Development and Evaluation using Mean Monthly Values
The following notebook will explore how a multiple linear regression can explore the monthly mean concentration of four pollutants with the mean weather data and the upstream and oil and gas activity data.

## Definition of Models
A set of features for each pollutant were selected based on the correlations shown in the heat map and the scatter plots.

**Y1: SO2**
X1: Temperature, X2: Wind speed, X3: Humidity, X4: Gas produced

**Y2: TRS**
X1: Wind speed, X2: Wind direction, X3: Depth drilled, X4: Gas produced

**Y3: NO2**
X1: Temperature, X2: Wind direction, X3: Wind speed, X4: Humidity, X5: Depth drilled, X6: Gas produced

**Y4: O3**
X1: Temperature, X2: Wind direction, X3: Wind speed, X4: Humidity, X5: Depth drilled, X6: Gas produced

### Data Splitting, Loss Functions & Cross-Validation

**Data Splitting**
The dataset will be split into a training and a testing dataset. The test dataset will include approximately 1/3 of the dataset to ensure that both the testing and the training sets are representative of the smaller-sized dataset.

**Feature Selection**
The correlation heat maps will be used to select the dependant variables. The features will not be scaled as these are not large datasets, and therefore the time to converge is not a concern.

**Loss Functions**
The sklearn LinearRegression fits a linear model that minimizes the Root Mean Squared Error (RMSE) loss function. This loss function cannot be changed. This loss function has the model learn the outlier data but applies high penalties to incorrect predictions on outlier data points.

**Cross Validation**
The train-validation-test split will not be used. The model is working with a smaller dataset. Therefore, the data will only be split into a training and a testing set. By introducing a validation set, the smaller size of the training, validation, and testing set will be less representative of the sample data.
A Grid Search Cross-Validation will be used. This grid will explore different parameters for the Linear Regression, including the intercept and is the regressors X will be normalized before regression. A 5-fold split will be used in this grid search to cross validate.

**Evaluation & Testing**
The model will be evaluated using the R^2^ score. The R^2^ measures the proportion of variance in the dependant variable that is predictable from the independent variables and is commonly used for linear regression. A higher R^2^ value indicates a better fit. Residual plots will also be used to ensure no transformations are required for the data and a linear model was the best choice (versus a polynomial).

**Other Possible Models for Future Consideration**
Sklearn offers another linear regression: the Huber Regressor that is a linear regression more robust to outliers and the Lasso model that that combines L1 and L2 loss functions.

In [ ]:
# Import third party libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import linear_model as lm
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from scipy.stats import circmean
import os

In [ ]:
# Read in monthly df
month_mean = pd.read_csv('month_mean.csv')

## Model 1: SO2

In [ ]:
# Split df into x and y
X =  month_mean.filter(['TEMP_MEAN_(C)', 'WSPD_SCLR_(M/S)', 'HUMIDITY_(%)', 'gas_prod_vol_m3'], axis=1)
y = month_mean.filter(['SO2'], axis=1)

# Split the data into a training and a testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

# Print test and trainning sizes
print('Train {}%'.format(X_train.shape[0] / month_mean.shape[0] * 100))
print('Test {}%'.format(X_test.shape[0] / month_mean.shape[0] * 100))

In [ ]:
# Create a LinearRegression object and name it linear_model
linear_model = lm.LinearRegression()

# Define parameters to explore
parameters = [{'fit_intercept':[True, False],
               'normalize':[True, False]}]

# Hyper-Parameter Tuning and Cross Validation
grid_search = GridSearchCV(estimator=linear_model,
                           param_grid=parameters,
                           scoring = 'r2',
                           cv = 5,
                           verbose =0,
                           return_train_score=True)
# Fit model to training data
grid_search.fit(X_train, y_train)

# Predict training data y using model
y_fitted = grid_search.predict(X_train)

# Print the r-squared value
print(r2_score(y_train, y_fitted))

In [ ]:
# Plot the residuals
residuals = y_train - y_fitted
ax = sns.regplot(x=y_train, y=residuals['SO2'])
ax.set_xlabel('SO2 (training data)')
ax.set_ylabel('Residuals (Actual SO2 - Predicted SO2')
plt.title('Residual Plot for SO2 Training Data')
plt.show()

In [ ]:
# Now predict the test values and calculate the r-squared score
y_fitted = grid_search.predict(X_test)
print(r2_score(y_test, y_fitted))

## Model 2: TRS

In [ ]:
# Split df into x and y
X =  month_mean.filter(['WSPD_SCLR_(M/S)', 'WDIR_VECT_(DEG)', 'Depth_per_day', 'gas_prod_vol_m3'], axis=1)
y = month_mean.filter(['TRS'], axis=1)

# Split the data into a training and a testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

# Print test and trainning sizes
print('Train {}%'.format(X_train.shape[0] / month_mean.shape[0] * 100))
print('Test {}%'.format(X_test.shape[0] / month_mean.shape[0] * 100))

In [ ]:
# Create a LinearRegression object and name it linear_model
linear_model = lm.LinearRegression()

# Define parameters to explore
parameters = [{'fit_intercept':[True, False],
               'normalize':[True, False]}]

# Hyper-Parameter Tuning and Cross Validation
grid_search = GridSearchCV(estimator=linear_model,
                           param_grid=parameters,
                           scoring = 'r2',
                           cv = 5,
                           verbose =0,
                           return_train_score=True)
# Fit model to training data
grid_search.fit(X_train, y_train)

# Predict training data y using model
y_fitted = grid_search.predict(X_train)

# Print the r-squared value
print(r2_score(y_train, y_fitted))

In [ ]:
# Plot the residuals
residuals = y_train - y_fitted
ax = sns.regplot(x=y_train, y=residuals['TRS'])
ax.set_xlabel('TRS (training data)')
ax.set_ylabel('Residuals (Actual TRS - Predicted TRS')
plt.title('Residual Plot for TRS Training Data')
plt.show()

In [ ]:
# Now predict the test values and calculate the r-squared score
y_fitted = grid_search.predict(X_test)
print(r2_score(y_test, y_fitted))

## Model 3: NO2

In [ ]:
# Split df into x and y
X =  month_mean.filter(['TEMP_MEAN_(C)', 'WDIR_VECT_(DEG)', 'WSPD_SCLR_(M/S)', 'HUMIDITY_(%)', 'Depth_per_day','gas_prod_vol_m3'], axis=1)
y = month_mean.filter(['NO2'], axis=1)

# Split the data into a training and a testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=10)

# Print test and trainning sizes
print('Train {}%'.format(X_train.shape[0] / month_mean.shape[0] * 100))
print('Test {}%'.format(X_test.shape[0] / month_mean.shape[0] * 100))

In [ ]:
# Create a LinearRegression object and name it linear_model
linear_model = lm.LinearRegression()

# Define parameters to explore
parameters = [{'fit_intercept':[True, False],
               'normalize':[True, False]}]

# Hyper-Parameter Tuning and Cross Validation
grid_search = GridSearchCV(estimator=linear_model,
                           param_grid=parameters,
                           scoring = 'r2',
                           cv = 5,
                           verbose =0,
                           return_train_score=True)
# Fit model to training data
grid_search.fit(X_train, y_train)

# Predict training data y using model
y_fitted = grid_search.predict(X_train)

# Print the r-squared value
print(r2_score(y_train, y_fitted))

In [ ]:
# Plot the residuals
residuals = y_train - y_fitted
ax = sns.regplot(x=y_train, y=residuals['NO2'])
ax.set_xlabel('NO2 (training data)')
ax.set_ylabel('Residuals (Actual NO2 - Predicted NO2')
plt.title('Residual Plot for NO2 Training Data')
plt.show()

In [ ]:
# Now predict the test values and calculate the r-squared score
y_fitted = grid_search.predict(X_test)
print(r2_score(y_test, y_fitted))

## Model 4: O3

In [ ]:
# Split df into x and y
X =  month_mean.filter(['TEMP_MEAN_(C)', 'WDIR_VECT_(DEG)', 'WSPD_SCLR_(M/S)', 'HUMIDITY_(%)','gas_prod_vol_m3'], axis=1)
y = month_mean.filter(['O3'], axis=1)

# Split the data into a training and a testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

# Print test and trainning sizes
print('Train {}%'.format(X_train.shape[0] / month_mean.shape[0] * 100))
print('Test {}%'.format(X_test.shape[0] / month_mean.shape[0] * 100))

In [ ]:
# Create a LinearRegression object and name it linear_model
linear_model = lm.LinearRegression()

# Define parameters to explore
parameters = [{'fit_intercept':[True, False],
               'normalize':[True, False]}]

# Hyper-Parameter Tuning and Cross Validation
grid_search = GridSearchCV(estimator=linear_model,
                           param_grid=parameters,
                           scoring = 'r2',
                           cv = 5,
                           verbose =0,
                           return_train_score=True)
# Fit model to training data
grid_search.fit(X_train, y_train)

# Predict training data y using model
y_fitted = grid_search.predict(X_train)

# Print the r-squared value
print(r2_score(y_train, y_fitted))

In [ ]:
# Plot the residuals
residuals = y_train - y_fitted
ax = sns.regplot(x=y_train, y=residuals['O3'])
ax.set_xlabel('O3 (training data)')
ax.set_ylabel('Residuals (Actual O3 - Predicted O3')
plt.title('Residual Plot for O3 Training Data')
plt.show()

In [ ]:
# Now predict the test values and calculate the r-squared score
y_fitted = grid_search.predict(X_test)
print(r2_score(y_test, y_fitted))